In [154]:
import pandas as pd
from glob import glob
import pickle
from omegaconf import OmegaConf
import os
import numpy as np

search_pattern_report="./batch_1-2/*/test/*pkl"
search_pattern_config="./batch_1-2/*/config.yaml"


# load the reports
exp_reports = dict()
for report_fp in glob(search_pattern_report):
    # recover experiment name from path
    n = [n for n in report_fp.split(os.sep) if '_v' in n][0]
    print(n)
    with open(report_fp, 'rb') as f:
        report = pickle.load(f)
        exp_reports.update({n:report})
configs = [OmegaConf.load(cfg_fp) for cfg_fp in glob(search_pattern_config)]
assert len(configs) == len(exp_reports)

2024-12-16_01-19-40_v002
2024-12-16_02-12-37_v003
2024-12-16_03-07-36_v005
2024-12-16_04-14-52_v000
2024-12-16_05-25-50_v001
2024-12-16_06-32-56_v004
2024-12-16_15-22-33_v005
2024-12-16_16-04-25_v006
2024-12-16_16-55-20_v007


['mIoU@L3', 'mAcc@L3']

In [117]:
list(range(len(columns_overview)-1))

[0, 1]

In [125]:
df_overview

,experiment_name,mIoU@L3,mAcc@L3


In [122]:
df

,experiment_name,mIoU@L3,mAcc@L3
0,2024-12-16_01-19-40_v002,None,None


In [183]:
cfg.data.scalar_input_fields + [k for k in cfg.model.keys() if 'ignore_' in k]

['xyz', 'rgb', 'riegl_feats', 'geom_feats', 'normals', 'incAngles', 'distanceFromScanner', 'ignore_waveform', 'ignore_neibors']

In [240]:

all_sifs = ['xyz', 'rgb', 'riegl_feats', 'geom_feats', 'normals', 'incAngles', 'distanceFromScanner', 'raw waveform', 'process neibors']
columns_overview = [
    'experiment_name',
] + all_sifs \
  + [n for nn in [[f'mIoU@{l.replace('labels_','L')}', f'mAcc@{l.replace('labels_','L')}'] for l in configs[0].data.label_names] for n in nn]

cat_names = list(configs[0].data.label_schema['labels_3'].keys()) # FIXME: only works if labels_3 is the level

columns_classwise = ['experiment_name']  + cat_names

df_overview = pd.DataFrame(columns=columns_overview)

# levels = list(list(reports.values())[0].keys())
for cfg, (exp_name, reports) in zip(configs, exp_reports.items()):
    # print("*"*(len(exp_name)+4))
    # print(f"| {exp_name} |")
    # print("*"*(len(exp_name)+4))
    df_exp_overview = pd.DataFrame(data=([[exp_name]+[None for _ in range(len(columns_overview)-1)]]), columns=columns_overview)
    miou, ious ,macc = [] , [], []
    metrics = {n:dict(
        miou=list(),
        ious=list(),
        macc=list(),
        pc_acc=list()
        
    ) for n in cfg.data.label_names}
    for proj_name, report in reports.items():
        # print(proj_name)
        for level in cfg.data.label_names:
            metrics[level]['miou'].append(report[level]['miou'])
            metrics[level]['ious'].append(report[level]['iou'])
            metrics[level]['macc'].append(report[level]['macc'])
            metrics[level]['pc_acc'].append(report[level]['pc_acc'])
            # print(f'\t\t{level}: mIoU: {report[level]['miou']*100:.2f}\tmAcc:{report[level]['macc']*100:.2f}')
            # print('\n')
    # raise NotImplementedError
    for level in cfg.data.label_names:
        # overall
        miou = np.mean(metrics[level]['miou'])*100
        macc = np.mean(metrics[level]['macc'])*100
        df_exp_overview[f'mIoU@L{level.replace('labels_','')}'] = miou
        df_exp_overview[f'mAcc@L{level.replace('labels_','')}'] = macc
        
        # class-wise
        
        
    
    # add config values
    
    for sif in all_sifs:
        if sif in cfg.data.scalar_input_fields:
            df_exp_overview[sif] = r'\checkmark'
        else:
            df_exp_overview[sif] = ""
            
    # handle ignores
    df_exp_overview['raw waveform'] = r'\checkmark' if not(cfg.model.ignore_waveform) else ''
    df_exp_overview['process neibors'] = r'\checkmark' if not(cfg.model.ignore_neibors) else ''
    # concat
    df_overview = pd.concat([df_overview, df_exp_overview],ignore_index=True)
    
    
# df_overview = df_overview.sort_values(by=df_overview['experiment_name'].apply(lambda x: x.split("_")[-1]))
    # TODO: Perform Averiging
    df_overview['version'] = df_overview['experiment_name'].apply(lambda x: x.split('_')[-1])
    df_overview = df_overview.sort_values(by='version').drop(columns='version').reset_index(drop=True)

# rename experiments
df_overview['experiment_name'] = df_overview['experiment_name'].apply(lambda x: x.split('_')[-1].replace('v',r'\#'))
# rename columns
df_overview = df_overview.rename(columns={
    'experiment_name' : '',
    'xyz' : 'XYZ',
    'rgb' : 'RGB',
    'riegl_feats' : 'Riegl',
    'geom_feats': 'Geom',
    'normals':r'$\[n_x,n_y,n_z\]$',
    'distanceFromScanner':r'$D_\text{scanner}$'
})


C:\Users\Hristo\AppData\Local\Temp\ipykernel_32856\2532516172.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_overview = pd.concat([df_overview, df_exp],ignore_index=True)


In [245]:
df_overview = df_overview.drop([6])

In [246]:
df_overview

,,XYZ,RGB,Riegl,Geom,"$\[n_x,n_y,n_z\]$",incAngles,$D_\text{scanner}$,raw waveform,process neibors,mIoU@L3,mAcc@L3
0,\#000,\checkmark,,,,,,,,\checkmark,3.726174,8.673848
1,\#001,\checkmark,\checkmark,,,,,,,\checkmark,11.743101,29.371643
2,\#002,\checkmark,\checkmark,\checkmark,,,,,,\checkmark,16.362155,48.114046
3,\#003,\checkmark,\checkmark,\checkmark,,,\checkmark,\checkmark,\checkmark,\checkmark,16.977851,40.188612
4,\#004,\checkmark,\checkmark,,,,\checkmark,\checkmark,\checkmark,\checkmark,15.142054,34.512015
5,\#005,\checkmark,\checkmark,,,,,,\checkmark,\checkmark,12.908594,31.485221
7,\#006,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,20.513423,56.818999
8,\#007,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,\checkmark,,3.726174,8.673848


In [247]:
print(df_overview.to_latex(index=False, float_format="%.2f"))

\begin{tabular}{llllllllllrr}
\toprule
 & XYZ & RGB & Riegl & Geom & $\[n_x,n_y,n_z\]$ & incAngles & $D_\text{scanner}$ & raw waveform & process neibors & mIoU@L3 & mAcc@L3 \\
\midrule
\#000 & \checkmark &  &  &  &  &  &  &  & \checkmark & 3.73 & 8.67 \\
\#001 & \checkmark & \checkmark &  &  &  &  &  &  & \checkmark & 11.74 & 29.37 \\
\#002 & \checkmark & \checkmark & \checkmark &  &  &  &  &  & \checkmark & 16.36 & 48.11 \\
\#003 & \checkmark & \checkmark & \checkmark &  &  & \checkmark & \checkmark & \checkmark & \checkmark & 16.98 & 40.19 \\
\#004 & \checkmark & \checkmark &  &  &  & \checkmark & \checkmark & \checkmark & \checkmark & 15.14 & 34.51 \\
\#005 & \checkmark & \checkmark &  &  &  &  &  & \checkmark & \checkmark & 12.91 & 31.49 \\
\#006 & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & 20.51 & 56.82 \\
\#007 & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \checkmark & \che

In [162]:
df_overview['experiment_name'].apply(lambda x: x.split("_")[-1])

0    v002
1    v003
2    v005
3    v000
4    v001
5    v004
6    v005
7    v006
8    v007
Name: experiment_name, dtype: object

In [159]:
df_overview

,experiment_name,mIoU@L3,mAcc@L3
0,2024-12-16_01-19-40_v002,0.163622,0.481140
1,2024-12-16_02-12-37_v003,0.169779,0.401886
2,2024-12-16_03-07-36_v005,0.129086,0.314852
3,2024-12-16_04-14-52_v000,0.037262,0.086738
4,2024-12-16_05-25-50_v001,0.117431,0.293716
5,2024-12-16_06-32-56_v004,0.151421,0.345120
6,2024-12-16_15-22-33_v005,0.127630,0.299709
7,2024-12-16_16-04-25_v006,0.205134,0.568190
8,2024-12-16_16-55-20_v007,0.037262,0.086738


In [142]:
np.set_printoptions(suppress=True)
np.nanmean(np.array(metrics['labels_3']['ious']),axis=0)


array([0.0003083 , 0.31987058, 0.        , 0.        , 0.48117412,
       0.        , 0.36869524, 0.05389096, 0.        , 0.33852922,
       0.        , 0.28081915])

In [127]:

np.mean(metrics['labels_3']['miou'])

0.16362154677992727

In [33]:
report[level]

{'iou': array([0.        , 0.        ,        nan, 0.        , 0.54096119,
        0.        , 0.        , 0.        ,        nan,        nan,
        0.        , 0.        ]),
 'pc_acc': array([0.        , 0.        ,        nan, 0.        , 1.17846504,
        0.        , 0.        , 0.        ,        nan,        nan,
        0.        , 0.        ]),
 'miou': 0.060106798925996144,
 'macc': 0.13094056027579887}